In [1]:
import numpy as np


In [6]:
import numpy as np

class ConvLayer:
    """
    A layer to perform convolution with multiple filters.
    """
    def __init__(self, num_filters, filter_size):
        # num_filters: Number of kernels
        # filter_size: An integer (e.g., 3 for a 3x3 filter)

        self.num_filters = num_filters
        self.filter_size = filter_size

        # Initialize random filters. The depth is 1 for grayscale images.
        # The weights are initialized randomly, in a real scenario they are learned.
        self.filters = np.random.randn(num_filters, filter_size, filter_size) / (filter_size * filter_size)

    def convolve2d(self, image, kernel):
        """Helper function for a single 2D convolution."""
        k_h, k_w = kernel.shape
        img_h, img_w = image.shape

        # Calculate output dimensions (assuming stride=1, padding=0)
        out_h = img_h - k_h + 1
        out_w = img_w - k_w + 1

        output = np.zeros((out_h, out_w))

        for y in range(out_h):
            for x in range(out_w):
                roi = image[y:y + k_h, x:x + k_w]
                output[y, x] = np.sum(roi * kernel)

        return output

    def forward(self, input_image):
        """
        Performs a forward pass of the conv layer.

        Args:
            input_image (np.array): A 2D numpy array.

        Returns:
            np.array: A 3D array of feature maps.
        """
        # We store the input for backpropagation later (not implemented here)
        self.last_input = input_image

        height, width = input_image.shape

        # Calculate output dimensions
        output_height = height - self.filter_size + 1
        output_width = width - self.filter_size + 1

        feature_maps = np.zeros((self.num_filters, output_height, output_width))

        # Apply each filter to the input image
        for i in range(self.num_filters):
            feature_maps[i] = self.convolve2d(input_image, self.filters[i])

        return feature_maps




class ReLU:
    """ReLU activation function layer."""
    def forward(self, input_data):
        # Apply ReLU element-wise
        self.last_input = input_data
        return np.maximum(0, input_data)

class MaxPoolingLayer:
    """
    A layer to perform max pooling.
    """
    def __init__(self, pool_size=2, stride=2):
        self.pool_size = pool_size
        self.stride = stride

    def forward(self, feature_maps):
        """
        Downsamples the feature maps.

        Args:
            feature_maps (np.array): A 3D array from the conv layer.

        Returns:
            np.array: A downsampled 3D array.
        """
        self.last_input = feature_maps
        num_filters, input_h, input_w = feature_maps.shape

        # Calculate output dimensions
        output_h = (input_h - self.pool_size) // self.stride + 1
        output_w = (input_w - self.pool_size) // self.stride + 1

        downsampled_maps = np.zeros((num_filters, output_h, output_w))

        for i in range(num_filters):
            for y in range(output_h):
                for x in range(output_w):
                    # Define the pooling window
                    y_start = y * self.stride
                    y_end = y_start + self.pool_size
                    x_start = x * self.stride
                    x_end = x_start + self.pool_size

                    # Extract the window and find the max value
                    window = feature_maps[i, y_start:y_end, x_start:x_end]
                    downsampled_maps[i, y, x] = np.max(window)

        return downsampled_maps



class FlattenLayer:
    """Flattens the output of the pooling layer into a 1D array."""
    def forward(self, input_data):
        self.last_input_shape = input_data.shape
        # Flatten the multi-dimensional input into a 1D vector
        return input_data.flatten()

class DenseLayer:
    """
    A fully connected layer for the ANN part.
    """
    def __init__(self, input_size, output_size):
        # output_size is the number of classes (e.g., 10 for digits 0-9)
        self.weights = np.random.randn(input_size, output_size) * 0.1
        self.biases = np.zeros(output_size)

    def forward(self, input_vector):
        """Performs the forward pass for the dense layer."""
        self.last_input = input_vector
        # Standard ANN calculation: input * weights + biases
        return np.dot(input_vector, self.weights) + self.biases


In [7]:
# --- 1. Define a sample input image (e.g., a 10x10 grayscale image) ---
sample_image = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 255, 255, 255, 255, 255, 255, 0, 0],
    [0, 0, 255, 0, 0, 0, 0, 255, 0, 0],
    [0, 0, 255, 0, 0, 0, 0, 255, 0, 0],
    [0, 0, 255, 0, 0, 0, 0, 255, 0, 0],
    [0, 0, 255, 255, 255, 255, 255, 255, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])

# --- 2. Initialize the layers of our network ---
conv = ConvLayer(num_filters=8, filter_size=3) # 8 filters, each 3x3
relu = ReLU()
pool = MaxPoolingLayer(pool_size=2)
flatten = FlattenLayer()

# The input size for the dense layer depends on the output of the pooling layer
# Conv output: 10-3+1 = 8x8. With 8 filters -> (8, 8, 8)
# Pool output: (8-2)/2+1 = 4x4. With 8 filters -> (8, 4, 4)
# Flattened size: 8 * 4 * 4 = 128
dense = DenseLayer(input_size=128, output_size=10) # 10 output classes

# --- 3. Run the forward pass step-by-step ---
print(f"Input image shape: {sample_image.shape}\n")

# Pass through Convolutional Layer
feature_maps = conv.forward(sample_image)
print(f"After Conv Layer (feature maps shape): {feature_maps.shape}\n")

# Pass through ReLU Activation
activated_maps = relu.forward(feature_maps)
# Shape doesn't change here

# Pass through Max Pooling Layer
pooled_maps = pool.forward(activated_maps)
print(f"After Max Pooling Layer (downsampled maps shape): {pooled_maps.shape}\n")

# Flatten the result
flat_vector = flatten.forward(pooled_maps)
print(f"After Flatten Layer (vector length): {flat_vector.shape[0]}\n")

# Feed into the ANN Dense Layer
final_output = dense.forward(flat_vector)
print(f"Final ANN Output (logits for 10 classes):\n {final_output}")

# To get final probabilities, you would apply a Softmax function
def softmax(logits):
    exps = np.exp(logits - np.max(logits)) # Subtract max for numerical stability
    return exps / np.sum(exps)

final_probabilities = softmax(final_output)
predicted_class = np.argmax(final_probabilities)

print(f"\nFinal Probabilities:\n {np.round(final_probabilities, 3)}")
print(f"\nPredicted Class: {predicted_class}")

Input image shape: (10, 10)

After Conv Layer (feature maps shape): (8, 8, 8)

After Max Pooling Layer (downsampled maps shape): (8, 4, 4)

After Flatten Layer (vector length): 128

Final ANN Output (logits for 10 classes):
 [  65.6665202   -42.26949394   90.79486478  -77.063693     -8.16801771
  -18.14976095 -132.2134485   -19.93759681  -20.88200213   98.68954855]

Final Probabilities:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]

Predicted Class: 9
